## 嵌入

在我們之前的示例中，我們對長度為“vocab_size”的高維詞袋向量進行了操作，並且我們明確地將低維位置表示向量轉換為稀疏的 one-hot 表示。 這種 one-hot 表示不是內存有效的，此外，每個單詞都是相互獨立處理的，即 one-hot 編碼的向量不表示單詞之間的任何語義相似性。

在本單元中，我們將繼續探索 **News AG** 數據集。 首先，讓我們加載數據並從上一個單元中獲取一些定義。


In [6]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...
Building vocab...
Vocab size =  95812


### 什麼是嵌入？

**embedding** 的思想是用低維密集向量來表示單詞，以某種方式反映單詞的語義。我們稍後將討論如何構建有意義的詞嵌入，但現在讓我們將嵌入視為一種降低詞向量維數的方法。

因此，嵌入層會將一個詞作為輸入，並產生一個指定“embedding_size”的輸出向量。從某種意義上說，它與“線性”層非常相似，但它不是採用單熱編碼向量，而是能夠將單詞編號作為輸入。

通過使用嵌入層作為我們網絡的第一層，我們可以從 bag-or-words 切換到 **embedding bag** 模型，我們首先將文本中的每個單詞轉換為相應的嵌入，然後計算一些聚合函數所有這些嵌入，例如“sum”、“average”或“max”。

![顯示五個序列詞的嵌入分類器的圖像。](./img/embedding-classifier-example.png)

我們的分類器神經網絡將從嵌入層開始，然後是聚合層，最後是線性分類器：

In [7]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### 處理可變序列大小

由於這種架構，我們網絡的小批量需要以某種方式創建。 在前面的單元中，當使用詞袋時，無論文本序列的實際長度如何，小批量中的所有 BoW 張量都具有相同的大小“vocab_size”。 一旦我們轉向詞嵌入，我們最終會在每個文本樣本中得到可變數量的詞，當將這些樣本組合成小批量時，我們將不得不應用一些填充。

這可以使用向數據源提供 `collate_fn` 函數的相同技術來完成：


In [8]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### 訓練嵌入分類器

現在我們已經定義了合適的數據加載器，我們可以使用我們在上一單元中定義的訓練函數來訓練模型：

In [9]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.648125
6400: acc=0.68671875
9600: acc=0.7127083333333334
12800: acc=0.730078125
16000: acc=0.738
19200: acc=0.7442708333333333
22400: acc=0.7525892857142857


(0.9628864856995761, 0.7588771593090211)

> **注意**：為了節省時間，我們這裡只訓練了 25k 條記錄（少於一個完整的 epoch），但是你可以繼續訓練，寫一個函數來訓練幾個 epoch，並用學習率參數來實驗 達到更高的精度。 你應該能夠達到大約 90% 的準確率。

### EmbeddingBag 層和可變長度序列表示

在之前的架構中，我們需要將所有序列填充到相同的長度，以便將它們放入一個小批量中。這不是表示可變長度序列的最有效方法 - 另一種方法是使用 **offset** 向量，它將保存存儲在一個大向量中的所有序列的偏移量。

![顯示偏移序列表示的圖像](./img/offset-sequence-representation.png)

> **注意**：在上圖中，我們顯示了一個字符序列，但在我們的示例中，我們使用的是單詞序列。然而，用偏移向量表示序列的一般原理保持不變。

為了使用偏移表示，我們使用 [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) 層。它類似於“Embedding”，但它以內容向量和偏移向量作為輸入，還包括平均層，可以是“mean”、“sum”或“max”。

這是使用“EmbeddingBag”的修改後的網絡：

In [10]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

為了準備訓練數據集，我們需要提供一個轉換函數來準備偏移向量：

In [11]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

請注意，與之前所有示例不同的是，我們的網絡現在接受兩個參數：數據向量和偏移向量，它們的大小不同。 有趣的是，我們的數據加載器還為我們提供了 3 個值而不是 2 個：文本和偏移向量都作為特徵提供。 因此，我們需要稍微調整我們的訓練函數來解決這個問題：

In [12]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.623125
6400: acc=0.675625
9600: acc=0.7028125
12800: acc=0.722890625
16000: acc=0.7355
19200: acc=0.7471354166666667
22400: acc=0.7539732142857143


(22.325298404510555, 0.7582773512476008)

## 語義嵌入：Word2Vec

在我們前面的例子中，模型嵌入層學會了將單詞映射到向量表示，然而，這種表示沒有太多的語義意義。學習這樣的向量表示會很好，類似的詞或符號將對應於在某些向量距離（例如歐幾里得距離）方面彼此接近的向量。

為此，我們需要以特定方式在大量文本上預訓練我們的嵌入模型。訓練語義嵌入的第一種方法稱為 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)。它基於兩個主要架構，用於生成單詞的分佈式表示：

 - **連續詞袋**（CBoW）——在這個架構中，我們訓練模型從周圍的上下文中預測一個詞。給定 ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$，模型的目標是從 $(W_{-2},W_{-1} ,W_1,W_2)$。
 - **Continuous skip-gram** 與 CBoW 相對。該模型使用上下文詞的周圍窗口來預測當前詞。

CBoW 更快，而 skip-gram 更慢，但在表示不常用的詞方面做得更好。

![顯示 CBoW 和 Skip-Gram 算法將單詞轉換為向量的圖像。](./img/example-algorithms-for-converting-words-to-vectors.png)

為了試驗在 Google 新聞數據集上預訓練的 word2vec 嵌入，我們可以使用 **gensim** 庫。下面我們找到與“神經”最相似的詞

> **注意：** 當您第一次創建詞向量時，下載它們可能需要一些時間！

In [13]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


我們還可以從單詞中提取向量嵌入，用於訓練分類模型（為了清晰起見，我們只顯示了向量的前 20 個分量）：

In [15]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

語義嵌入的好處在於您可以操縱向量編碼來更改語義。 例如，我們可以要求找到一個單詞，它的向量表示盡可能接近單詞 *king* 和 *woman*，並且遠離單詞 *man*：

In [16]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBOW 和 Skip-Grams 都是“預測性”嵌入，因為它們只考慮本地上下文。 Word2Vec 不利用全局上下文。

**FastText**，通過學習每個單詞的向量表示和在每個單詞中找到的字符 n-gram 來構建在 Word2Vec 上。 然後在每個訓練步驟中將表示的值平均為一個向量。 雖然這為預訓練增加了很多額外的計算，但它使詞嵌入能夠對子詞信息進行編碼。

另一種方法**GloVe**利用共現矩陣的思想，使用神經方法將共現矩陣分解為更具表現力的非線性詞向量。

您可以通過將嵌入更改為 FastText 和 GloVe 來使用示例，因為 gensim 支持

## 在 PyTorch 中使用預訓練嵌入

我們可以修改上面的示例，以使用語義嵌入（例如 Word2Vec）預先填充嵌入層中的矩陣。 我們需要考慮到預訓練嵌入的詞彙表和我們的文本語料庫可能不匹配，因此我們將使用隨機值初始化缺失詞的權重：

In [17]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.itos):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


現在讓我們訓練我們的模型。 請注意，由於更大的嵌入層尺寸以及更多的參數，訓練模型所需的時間明顯大於前一個示例。 此外，因此，如果我們想避免過度擬合，我們可能需要在更多示例上訓練我們的模型。

In [18]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6384375
6400: acc=0.68359375
9600: acc=0.7092708333333333
12800: acc=0.727734375
16000: acc=0.7389375
19200: acc=0.7481770833333333
22400: acc=0.7566517857142857


(221.17328454894434, 0.7620361484325016)

在我們的例子中，我們沒有看到準確度的巨大提高，這可能是完全不同的聲音。
為了克服不同詞彙的問題，我們可以使用以下解決方案之一：
* 根據我們的詞彙重新訓練 word2vec 模型
* 使用預訓練的 word2vec 模型中的詞彙加載我們的數據集。 可以在加載期間指定用於加載數據集的詞彙。

後一種方法似乎更容易，特別是因為 PyTorch `torchtext` 框架包含對嵌入的內置支持。 例如，我們可以通過以下方式實例化基於 GloVe 的詞彙表：

In [19]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [05:01, 2.86MB/s]                           
100%|█████████▉| 399999/400000 [00:07<00:00, 52389.65it/s]


加載的詞彙有以下基本操作：
* `vocab.stoi` 字典允許我們將單詞轉換為其字典索引
* `vocab.itos` 做相反的事情 - 將數字轉換為單詞
* `vocab.vectors` 是嵌入向量的數組，因此要獲得單詞 `s` 的嵌入，我們需要使用 `vocab.vectors[vocab.stoi[s]]`

這是操縱嵌入的示例以演示等式 **kind-man+woman = Queen**（我不得不稍微調整係數以使其起作用）：

In [20]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

要使用這些嵌入訓練分類器，我們首先需要使用 GloVe 詞彙對我們的數據集進行編碼：

In [21]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

正如我們在上面看到的，所有向量嵌入都存儲在 `vocab.vectors` 矩陣中。 使用簡單的複制將這些權重加載到嵌入層的權重中變得非常容易：

In [22]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Now let's train our model and see if we get better results:

In [23]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6296875
6400: acc=0.68796875
9600: acc=0.7104166666666667
12800: acc=0.725
16000: acc=0.7355
19200: acc=0.7427604166666667
22400: acc=0.7507142857142857


(37.088364223448494, 0.7558781190019194)

我們沒有看到準確率顯著提高的原因之一是由於我們數據集中的某些單詞在預訓練的 GloVe 詞彙表中缺失，因此它們基本上被忽略了。 為了克服這個事實，我們可以在我們的數據集上訓練我們自己的嵌入。


## 訓練你自己的嵌入

在我們的示例中，我們一直在使用預訓練的語義嵌入，但看看如何使用 CBoW 或 Skip-gram 架構訓練這些嵌入很有趣。 這個練習超出了這個模塊，但那些有興趣的人可能想看看這個[關於語言建模的官方 PyTorch 教程](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)。 此外，**gensim** 框架可用於在幾行代碼中訓練最常用的嵌入，如本文檔中所述（https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html） .

## 上下文嵌入

Word2Vec 等傳統預訓練嵌入表示的一個關鍵限制是詞義消歧問題。雖然預訓練嵌入可以捕獲上下文中單詞的某些含義，但單詞的每個可能含義都被編碼到相同的嵌入中。這可能會導致下游模型出現問題，因為許多單詞（例如“play”）根據使用的上下文具有不同的含義。

例如，這兩個不同句子中的“play”一詞具有完全不同的含義：
- 我去劇院看戲了。
- 約翰想和他的朋友們**玩**。

上面的預訓練嵌入在同一個嵌入中代表了單詞“play”的這兩種含義。為了克服這個限制，我們需要基於 **語言模型** 構建嵌入，該模型在大型文本語料庫上訓練，並且*知道*如何將單詞放在不同的上下文中。討論上下文嵌入超出了本教程的範圍，但我們將在下一個單元中討論語言模型時回到它們。
